# Investigating the correlation between plant pairs (related by genus) and their geolocation 

### Import modules

In [1]:
import logging
from collections import defaultdict
import json
import pandas as pd
from tqdm import tqdm
    
# Stats
from scipy.stats import pearsonr
from statannot import add_stat_annotation
import ptitprince as pt

# Viz
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

/Users/danieldomingo/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
%matplotlib inline

In [3]:
logging.disable()
pd.options.mode.chained_assignment = None

In [4]:
DATA_DIR = '../data/processed'

### Load plant-disease evidences

In [19]:
literature_pairs = pd.read_csv(
    f'{DATA_DIR}/database_based_correlation_data.tsv.gz',
    compression='gzip',
    sep='\t',
    names=['plant_1', 'plant_2', 'pair_type', 'pearson_coefficient'],
)

<ipython-input-19-3056a9de1951>:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  literature_pairs = pd.read_csv(


In [20]:
literature_pairs_genus.columns


Index(['plant_1', 'plant_2', 'pair_type', 'pearson_coefficient', 'cosine_sim'], dtype='object')

In [6]:
literature_pairs_genus = literature_pairs[literature_pairs['pair_type'] == 'Same genus']

In [7]:
medicinal_plants_lit = set(literature_pairs_genus['plant_1'].tolist()).union(
    literature_pairs_genus['plant_2'].tolist()
)
len(medicinal_plants_lit)

1586

### Loading plant location data

In [8]:
ncbi_ids_trefle = json.load(open('../data/mapping/trefle_grounding.json'))

In [9]:
trefle_df = pd.read_csv(
    '../data/raw/trefle_species_dump.csv', 
    sep='\t', 
    usecols=['scientific_name', 'distributions']
)

In [10]:
trefle_df['scientific_name'] = trefle_df['scientific_name'].str.replace('?', '')
trefle_df['ncbi_ids'] = trefle_df['scientific_name'].map(lambda x: ncbi_ids_trefle[x][0])
trefle_df = trefle_df[trefle_df['ncbi_ids'].isin(medicinal_plants_lit)]
trefle_df.head(3)

<ipython-input-10-baba2640493b>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  trefle_df['scientific_name'] = trefle_df['scientific_name'].str.replace('?', '')


,scientific_name,distributions,ncbi_ids
29,Abutilon theophrasti,"Thailand,China North-Central,China South-Centr...",ncbitaxon:3631
207,Achyranthes aspera,"Afghanistan,Andaman Is.,Angola,Assam,Banglades...",ncbitaxon:240005
208,Achyranthes aspera var. aspera,NaN,ncbitaxon:240005


In [11]:
geolocation_data = pd.read_csv(
    f'{DATA_DIR}/plant_geolocation.tsv.gz',
    compression='gzip',
    sep='\t', 
    index_col='Plant location (trefle)',
    usecols=['Plant location (trefle)', 'Country']
)
geolocation_data = geolocation_data[geolocation_data['Country'] != '-']

In [12]:
loc_info = geolocation_data.to_dict()['Country']
len(loc_info)

308

### Create location dictionary

In [13]:
plant_loc_data = defaultdict(set)
skip = set()

for _, locations, ncbi_ids in tqdm(trefle_df.values):
    if pd.isna(locations):
        continue
    for loc_name in locations.split(','):
        if loc_name not in loc_info:
            skip.add(loc_name)
            continue
        plant_loc_data[ncbi_ids].add(loc_info[loc_name])
        
len(plant_loc_data), len(skip)

100%|██████████| 2077/2077 [00:00<00:00, 49507.68it/s]


(1257, 59)

The locations that have been skipped in the previous step correspond to Islands.

### Correlation plot

In [14]:
def calculate_overlap_coefficient(elements_in_plant_1, elements_in_plant_2):
    """Calculate coeffiencient overlap."""
    if not elements_in_plant_1 or not elements_in_plant_2:
        return 0
    
    return len(
        elements_in_plant_1.intersection(elements_in_plant_2)
    ) / min([len(elements_in_plant_1), len(elements_in_plant_2)])

In [15]:
rows = []

for plant1, plant2, _, coeff, _ in tqdm(literature_pairs_genus.values):
    if plant1 not in plant_loc_data or plant2 not in plant_loc_data:
        continue
        
    loc_plant_1 = plant_loc_data[plant1]
    loc_plant_2 = plant_loc_data[plant2]
    overlap_coeff = calculate_overlap_coefficient(loc_plant_1, loc_plant_2)
    
    if overlap_coeff == 0:
        overlap_score = 'No overlap'
    elif overlap_coeff < 0.4:
        overlap_score = 'Low'
    elif 0.4 < overlap_coeff < 0.8:
        overlap_score = 'Medium'
    else:
        overlap_score = 'High'
        
    rows.append({
        'plant_1': plant1, 
        'plant_2': plant2,
        'Geographical similarity (country overlap)': overlap_score, 
        'pearson_coefficient': coeff,
    })

  0%|          | 0/3533 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 4)

In [ ]:
merged_df = pd.DataFrame(rows)

In [ ]:
sns.set(context='poster')
mpl.rcParams['figure.dpi'] = 400

In [ ]:
fig = plt.figure(figsize=(18, 11)) 
ax1 = fig.add_subplot(111)

suptitle = plt.suptitle(
    'Indication area similarity between \ngeographically-close and distant medicinal plants',
    y=0.99,
    fontweight='bold'
)

pt.RainCloud(
    data=merged_df,
    x='Geographical similarity (country overlap)',
    y='pearson_coefficient',
    ax=ax1,
    order=['No overlap', 'Low', 'Medium', 'High'],
)
add_stat_annotation(
    data=merged_df,
    x='Geographical similarity (country overlap)',
    y='pearson_coefficient',
    ax=ax1,
    box_pairs=[
        ("No overlap", "High"), ("No overlap", "Low"), ("Low", "Medium"), ("Medium", "High"), ("No overlap", "Medium"),
    ],
    test='Mann-Whitney',
    text_format='star', 
    line_offset=0,
    text_offset=0,
)

# Rotate ticks X
ax1.xaxis.set_tick_params(rotation=45)
ax1.set_ylabel("Pearson correlation")
    
plt.tight_layout()
plt.savefig(
    f'../data/figures/figure_3.png',
    dpi=400,
    bbox_inches='tight', 
    bbox_extra_artists=[suptitle],
)
plt.show()